In [14]:
import pandas as pd
import reverse_geocoder as rg

In [11]:
e_data = pd.read_csv("https://www.dropbox.com/s/u2xupuhf07y85zh/earthQuake.csv?dl=1")
density_data = pd.read_csv("https://www.dropbox.com/s/p6ejr4mzx8pjq9m/population_density_long.csv?dl=1")
population_data=pd.read_csv("https://www.dropbox.com/s/bp5hb61zrui2qo6/population_total_long.csv?dl=1")

separate the date into columns

In [4]:
e_data[['Month','Day','Year']]=e_data['Date'].str.split('/',-1,expand=True)

get rid of all unwanted columns (is Depth usefull?)

In [5]:
e_data = e_data.drop(['Date','Time','ID','Source','Location Source','Magnitude Source','Status','Type','Magnitude Type'],axis=1)

In [6]:
e_data.head()

,Latitude,Longitude,Depth,Magnitude,Month,Day,Year
0,19.246,145.616,131.6,6.0,01,02,1965
1,1.863,127.352,80.0,5.8,01,04,1965
2,-20.579,-173.972,20.0,6.2,01,05,1965
3,-59.076,-23.557,15.0,5.8,01,08,1965
4,11.938,126.427,15.0,5.8,01,09,1965


In [7]:
population_data.head()

,Country Name,Year,Count
0,Aruba,1960,54211
1,Afghanistan,1960,8996973
2,Angola,1960,5454933
3,Albania,1960,1608800
4,Andorra,1960,13411


In [8]:
density_data.head()

,Country Name,Year,Count
0,Aruba,1961,307
1,Afghanistan,1961,14
2,Angola,1961,4
3,Albania,1961,60
4,Andorra,1961,30


In [23]:
rg.search((1.863,127.352))

[{'lat': '1.49305',
  'lon': '127.55149',
  'name': 'Tongutisungi',
  'admin1': 'Maluku Utara',
  'admin2': '',
  'cc': 'ID'}]

In [17]:
type(density_data)

pandas.core.frame.DataFrame

In [28]:
a = density_data.loc[density_data['Country Name'] == 'Tongutisungi']

In [32]:
density_data.count()

Country Name    12067
Year            12067
Count           12067
dtype: int64

In [33]:
population_data.count()

Country Name    12595
Year            12595
Count           12595
dtype: int64